In [74]:
import pickle 
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from torch.utils.data import DataLoader, TensorDataset


In [75]:
# File paths (assuming these are correct as provided)
filepaths = {
    "encoders": "models/label_encoders.pkl",
    "scaler": "models/scaler.pkl",
    "linear_regression": "models/linear_model.pkl",
    "xgb_model": "models/xgb_model.pkl",
    "data": "models/dataframe.pkl",
    "neural_network": "models/nn_model.pth" 
}

# Load all necessary objects
with open(filepaths["encoders"], "rb") as f:
    encoders = pickle.load(f)

with open(filepaths["scaler"], "rb") as f:
    scaler = pickle.load(f)

with open(filepaths["linear_regression"], "rb") as f:
    lr_model = pickle.load(f)

with open(filepaths["xgb_model"], "rb") as f:
    xgb_model = pickle.load(f)

with open(filepaths["data"], "rb") as f:
    full_dataset = pickle.load(f)

print("\n✅ All objects loaded successfully!")


✅ All objects loaded successfully!


In [76]:
selected_cols = [
    "METRES_CADASTRE", "Energia primària no renovable", "Emissions de CO2",
    "Energia calefacció", "Energia refrigeració", "Energia calefacció demanda", "VALOR AILLAMENTS",
    "Cost anual aproximat d'energia per habitatge", "VALOR FINESTRES",
    "POBLACIO",  "Qualificació de consum d'energia primaria no renovable",
    "VEHICLE ELECTRIC", "SISTEMA BIOMASSA", "REHABILITACIO_ENERGETICA", 
    "Consum d'energia final"
    ]

numeric_features = [
    "METRES_CADASTRE", "Energia primària no renovable", "Emissions de CO2",
    "Energia calefacció", "Energia refrigeració", "Energia calefacció demanda", "VALOR AILLAMENTS",
    "Cost anual aproximat d'energia per habitatge", "VALOR FINESTRES"
    ]

target = "Consum d'energia final"

X_full = full_dataset[selected_cols].drop(target, axis=1)
y_full = full_dataset[target]

lr_model.fit(X_full, y_full)
xgb_model.fit(X_full, y_full)
print("\n✅ Models refitted with full dataset!")


✅ Models refitted with full dataset!


In [85]:
class EnergyNN(nn.Module):
    def __init__(self, input_dim):
        super(EnergyNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.output(x)
        return x
nn_model = EnergyNN(X_full.shape[1])  # Initialize with input dimension from full dataset
nn_model.load_state_dict(torch.load(filepaths["neural_network"]))
nn_model.eval()  # Set to evaluation mode



/var/folders/xz/5bn1vdj146n6586g9r465k_40000gn/T/ipykernel_96647/3719673932.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nn_model.load_state_dict(torch.load(filepath

EnergyNN(
  (fc1): Linear(in_features=14, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [120]:
# Example input
example_input = {
    'METRES_CADASTRE': 383, 
    'Energia primària no renovable': 308.355, 
    'Emissions de CO2': 40,
    'Energia calefacció': 5.25,
    'Energia refrigeració': 0,
    'Energia calefacció demanda': 5.25,
    'VALOR AILLAMENTS': 2.5,
    "Cost anual aproximat d'energia per habitatge": 2600,
    'VALOR FINESTRES': 3.0,
    'POBLACIO': 'Gelida',  
    "Qualificació de consum d'energia primaria no renovable": 'D',
    'VEHICLE ELECTRIC': '0', 
    'SISTEMA BIOMASSA': '0', 
    'REHABILITACIO_ENERGETICA': '0'
}

In [121]:
def preprocess_input(input_dict, encoders, scaler, reference_df):
    # Convert input to DataFrame
    input_df = pd.DataFrame([input_dict])
    
    # Separate numerical and categorical columns based on reference dataset
    numeric_features = [
    "METRES_CADASTRE", "Energia primària no renovable", "Emissions de CO2",
    "Energia calefacció", "Energia refrigeració", "Energia calefacció demanda", "VALOR AILLAMENTS",
    "Cost anual aproximat d'energia per habitatge", "VALOR FINESTRES"
    ]

# Categorical features (to encode)
    categorical_features = [
        "POBLACIO",  "Qualificació de consum d'energia primaria no renovable",
        "VEHICLE ELECTRIC", "SISTEMA BIOMASSA", "REHABILITACIO_ENERGETICA"
        ]     
    # Handle categorical variables
    for col in categorical_features:
        if col in input_df.columns:
            if col in encoders:
                try:
                    input_df[col] = encoders[col].transform(input_df[col])
                except ValueError as e:
                    print(f"Warning: Unseen label in {col}. Assigning default value...")
                    # Assign most common value or a neutral value from training data
                    default_value = reference_df[col].mode()[0]
                    input_df[col] = default_value
                    input_df[col] = encoders[col].transform([default_value])[0]
    
    # Ensure all columns match the training data
    input_df[numeric_features] = scaler.transform(input_df[numeric_features])
    
    # Scale numerical features
    input_scaled = input_df
    input_tensor = torch.tensor(input_scaled.values, dtype=torch.float32)
    
    return input_scaled, input_tensor


# Preprocess the input
processed_input, input_tensor = preprocess_input(example_input, encoders, scaler, X_full)


In [122]:
# Make predictions
lr_prediction = lr_model.predict(processed_input)
xgb_prediction = xgb_model.predict(processed_input)
with torch.no_grad():  # No gradient computation for inference
    nn_prediction = nn_model(input_tensor).item()  # Get scalar value from tensor

# Output results
print("\nPredictions:")
print(f"Linear Regression Prediction: {lr_prediction[0]:.2f} kWh/m²")
print(f"XGBoost Prediction: {xgb_prediction[0]:.2f} kWh/m²")
print(f"Neural Network Prediction: {nn_prediction:.2f} kWh/m²")


Predictions:
Linear Regression Prediction: 1.99 kWh/m²
XGBoost Prediction: 7.12 kWh/m²
Neural Network Prediction: 179.29 kWh/m²
